<a href="https://colab.research.google.com/github/miruna-morarasu/AI-Labs/blob/master/KI_Lab4_G11_hobbit_edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Collabs = [("Miruna-Maria Morarasu", "722/1"),
            ("Sonia Pojoga", "722/1"), 
            ("Dorel-Gabriel Posa", "722/1"), 
            ("Ovidiu-Dan Sonea", "722/2")]**



Dimensiunea matricii, respectiv a puzzle-ului:

In [0]:
n = 8

Modelarea unui puzzle n x n, sub forma clasei `Puzzle`, unde:

*   `pozx`, `pozY` = coordonatele pozitiei goale, marcate cu 0
*   `matrix` = reprezentare matriceala a configuratiei actuale a unui puzzle



In [0]:
class Puzzle:
    def __init__(self, matr, poz):
        self.pozX = poz // n
        self.pozY = poz % n
        self.matrix = [x[:n] for x in matr]

    def print_matrix(self):
        print(self.matrix)
# Costul pentru o piesa este numarul de pasi pana la pozitia corecta
# Din costul total il scadem pe cel pentru pozitia 0 (cea goala)
    def cost(self):
        to_return = 0
        for i in range(n):
            for j in range(n):
                to_return += (abs(self.matrix[i][j] - (i * n + j + 1))) % n + (abs(self.matrix[i][j] - (i * n + j + 1))) // n
        return to_return - ((abs(self.matrix[self.pozX][self.pozY] - (self.pozX * n + self.pozY + 1))) % n + (abs(self.matrix[self.pozX][self.pozY] - (self.pozX * n + self.pozY + 1))) // n)

    def move_up(self):
        if self.pozX != 0:
            aux = self.matrix[self.pozX][self.pozY]
            self.matrix[self.pozX][self.pozY] = self.matrix[self.pozX - 1][self.pozY]
            self.matrix[self.pozX - 1][self.pozY] = aux
            self.pozX -= 1
            return Puzzle(self.matrix, self.pozX * n + self.pozY)
        return None

    def move_down(self):
        if self.pozX != n - 1:
            aux = self.matrix[self.pozX][self.pozY]
            self.matrix[self.pozX][self.pozY] = self.matrix[self.pozX + 1][self.pozY]
            self.matrix[self.pozX + 1][self.pozY] = aux
            self.pozX += 1
            return Puzzle(self.matrix, self.pozX * n + self.pozY)
        return None

    def move_left(self):
        if self.pozY != 0:
            aux = self.matrix[self.pozX][self.pozY]
            self.matrix[self.pozX][self.pozY] = self.matrix[self.pozX][self.pozY - 1]
            self.matrix[self.pozX][self.pozY - 1] = aux
            self.pozY -= 1
            return Puzzle(self.matrix, self.pozX * n + self.pozY)
        return None

    def move_right(self):
        if self.pozY != n - 1:
            aux = self.matrix[self.pozX][self.pozY]
            self.matrix[self.pozX][self.pozY] = self.matrix[self.pozX][self.pozY + 1]
            self.matrix[self.pozX][self.pozY + 1] = aux
            self.pozY += 1
            return Puzzle(self.matrix, self.pozX * n + self.pozY)
        return None

    def __eq__(self, x):
        return self.matrix == x.matrix

`isSolvable()` - primeste un puzzle sub forma de array si returneaza:

*   `True` - daca acesta este rezolvabil
*   `False` - in caz contrar
> Metoda auxiliara: `getInvCount()`, folosita pentru a calcula numarul de inversiuni din cadrul array-ului


In [0]:
def getInvCount(arr):
    inv_count = 0
    for i in range(n * n - 1):
        for j in range(i + 1, n * n):
            if arr[j] and arr[i] and arr[i] > arr[j]:
                inv_count += 1
    return inv_count


def isSolvable(puzzle):
    invCount = getInvCount(puzzle)
    return (invCount % 2 == 0)

Modelarea unui table de sah corespunzatoare problemei damelor, varianta n x n, sub forma clasei `Damenspiel`, in care atributul `columns` reprezinta un vector de dimensiune n, in care:

*   indexul in lista corespunde randului de pe tabla de sah
*   elementul din lista corespunde coloanei de pe tabla de sah



In [0]:
class Damenspiel:
    def __init__(self, columns):
        self.columns = columns
# Costul reprezinta numarul de dame care se ataca
# Dorim un cost minim
    def cost(self):
        to_return = 0
        for i in range(n - 1):
            for j in range(i + 1, n):
                if self.columns[i] + i == self.columns[j] + j:
                    to_return += 1
                if abs(self.columns[i] - i) == abs(self.columns[j] - j):
                    to_return += 1
        return to_return

    def move(self):
        permutations = []
        for i in range(n - 1):
            for j in range(i + 1, n):
                perm = self.columns[:]
                perm[i] = self.columns[j]
                perm[j] = self.columns[i]
                permutations.append(Damenspiel(perm))
        return permutations

    def __eq__(self, x):
        return self.columns == x.columns

`bilbo()` - **Bergsteigeralgorithmus** (pentru Puzzle n x n) - primeste parametru o lista cu un obiect de tip `Puzzle`, reprezentand configuratia (random) initiala


In [0]:
def bilbo(node_list):
    step_count = 0
    stack = []
    visited = [Puzzle(node_list[0].matrix, node_list[0].pozX * n + node_list[0].pozY)]
    node_list.sort(key=lambda x: x.cost())
    while len(node_list) != 0 and node_list[0].cost() != 0:
        if step_count % 500 == 0:
            print(step_count, node_list[0].cost(), len(visited))
        possibi = []
        new_node = node_list[0].move_up()
        if new_node is not None and new_node.cost() <= node_list[0].cost() and new_node not in visited:
            possibi.append(new_node)
            node_list[0].move_down()
            visited.append(Puzzle(new_node.matrix, new_node.pozX * n + new_node.pozY))
        new_node = node_list[0].move_down()
        if new_node is not None and new_node.cost() <= node_list[0].cost() and new_node not in visited:
            possibi.append(new_node)
            node_list[0].move_up()
            visited.append(Puzzle(new_node.matrix, new_node.pozX * n + new_node.pozY))
        new_node = node_list[0].move_left()
        if new_node is not None and new_node.cost() <= node_list[0].cost() and new_node not in visited:
            possibi.append(new_node)
            node_list[0].move_right()
            visited.append(Puzzle(new_node.matrix, new_node.pozX * n + new_node.pozY))
        new_node = node_list[0].move_right()
        if new_node is not None and new_node.cost() <= node_list[0].cost() and new_node not in visited:
            possibi.append(new_node)
            node_list[0].move_left()
            visited.append(Puzzle(new_node.matrix, new_node.pozX * n + new_node.pozY))
        node_list.pop(0)
        possibi.sort(key=lambda x: x.cost())
        node_list = possibi + node_list
        step_count += 1
    return step_count

`kili()` - **Bergsteigeralgorithmus mit Random Restart** (pentru Puzzle n x n) - primeste parametru o lista cu un obiect de tip `Puzzle`, reprezentand configuratia (random) initiala


In [0]:
def kili(node_list):
    step_count = 0
    visited = [Puzzle(node_list[0].matrix, node_list[0].pozX * n + node_list[0].pozY)]
    node_list.sort(key=lambda x: x.cost())
    while len(node_list) != 0 and node_list[0].cost() != 0:
        if step_count % 500 == 0:
            print(step_count, node_list[0].cost(), len(visited))
        possibi = []
        new_node = node_list[0].move_up()
        if new_node is not None and new_node.cost() <= node_list[0].cost() and new_node not in visited:
            possibi.append(new_node)
            node_list[0].move_down()
            visited.append(Puzzle(new_node.matrix, new_node.pozX * n + new_node.pozY))
        new_node = node_list[0].move_down()
        if new_node is not None and new_node.cost() <= node_list[0].cost() and new_node not in visited:
            possibi.append(new_node)
            node_list[0].move_up()
            visited.append(Puzzle(new_node.matrix, new_node.pozX * n + new_node.pozY))
        new_node = node_list[0].move_left()
        if new_node is not None and new_node.cost() <= node_list[0].cost() and new_node not in visited:
            possibi.append(new_node)
            node_list[0].move_right()
            visited.append(Puzzle(new_node.matrix, new_node.pozX * n + new_node.pozY))
        new_node = node_list[0].move_right()
        if new_node is not None and new_node.cost() <= node_list[0].cost() and new_node not in visited:
            possibi.append(new_node)
            node_list[0].move_left()
            visited.append(Puzzle(new_node.matrix, new_node.pozX * n + new_node.pozY))
        if len(possibi) == 0:
            lista = random.sample(range(n * n), n * n)
            poz = lista.index(0)
            index = 0
            matr = []
            for i in range(n):
                matr.append([])
                for j in range(n):
                    matr[i].append(lista[index])
                    index += 1
            instance = Puzzle(matr, poz)
            while not isSolvable(lista) or instance.cost() > n * n:
                lista = random.sample(range(n * n), n * n)
                poz = lista.index(0)
                index = 0
                matr = []
                for i in range(n):
                    matr.append([])
                    for j in range(n):
                        matr[i].append(lista[index])
                        index += 1
                instance = Puzzle(matr, poz)
            node_list = [instance]
            print("restarted", step_count, node_list[0].cost())
        else:
            node_list.pop(0)
            possibi.sort(key=lambda x: x.cost())
            node_list = possibi + node_list
        step_count += 1
    return step_count

`fili()` - **Bergsteigeralgorithmus mit simulierten Abkühlung** (pentru Puzzle n x n) - primeste parametru o lista cu un obiect de tip `Puzzle`, reprezentand configuratia (random) initiala


In [0]:
def fili(node_list):
    step_count = 1
    visited = [Puzzle(node_list[0].matrix, node_list[0].pozX * n + node_list[0].pozY)]
    node_list.sort(key=lambda x: x.cost())
    while len(node_list) != 0 and node_list[0].cost() != 0:
        if step_count % 500 == 0:
            print(step_count, node_list[0].cost(), len(visited))
        temp = 1000 / step_count
        possibi = []
        new_node = node_list[0].move_up()
        if new_node is not None and (new_node.cost() <= node_list[0].cost() or random.uniform(0, 1) <= math.exp(
                abs(new_node.cost() - node_list[0].cost()) / temp)) and new_node not in visited:
            possibi.append(new_node)
            node_list[0].move_down()
            visited.append(Puzzle(new_node.matrix, new_node.pozX * n + new_node.pozY))
        new_node = node_list[0].move_down()
        if new_node is not None and (new_node.cost() <= node_list[0].cost() or random.uniform(0, 1) <= math.exp(
                abs(new_node.cost() - node_list[0].cost()) / temp)) and new_node not in visited:
            possibi.append(new_node)
            node_list[0].move_up()
            visited.append(Puzzle(new_node.matrix, new_node.pozX * n + new_node.pozY))
        new_node = node_list[0].move_left()
        if new_node is not None and (new_node.cost() <= node_list[0].cost() or random.uniform(0, 1) <= math.exp(
                abs(new_node.cost() - node_list[0].cost()) / temp)) and new_node not in visited:
            possibi.append(new_node)
            node_list[0].move_right()
            visited.append(Puzzle(new_node.matrix, new_node.pozX * n + new_node.pozY))
        new_node = node_list[0].move_right()
        if new_node is not None and (new_node.cost() <= node_list[0].cost() or random.uniform(0, 1) <= math.exp(
                abs(new_node.cost() - node_list[0].cost()) / temp)) and new_node not in visited:
            possibi.append(new_node)
            node_list[0].move_left()
            visited.append(Puzzle(new_node.matrix, new_node.pozX * n + new_node.pozY))
        node_list.pop(0)
        possibi.sort(key=lambda x: x.cost())
        node_list = possibi + node_list
        step_count += 1
    return step_count

`bofur()` - **Bergsteigeralgorithmus** (pentru Damespiel n x n) - primeste parametru o lista cu un obiect de tip `Damenspiel`, reprezentand configuratia (random) initiala


In [0]:
def bofur(node_list):
    step_count = 0
    visited = []
    node_list.sort(key=lambda x: x.cost())
    while len(node_list) != 0 and node_list[0].cost() != 0:
        visited.append(Damenspiel(node_list[0].columns))
        if step_count % 500 == 0:
            print(step_count, node_list[0].cost(), len(visited), node_list[0].columns)
        children = node_list[0].move()
        possibi = list(filter(lambda x: x.cost() <= node_list[0].cost() and x not in visited, children))
        possibi.sort(key=lambda x: x.cost())
        node_list.pop(0)
        node_list = possibi + node_list
        step_count += 1
    return step_count

`bombur()` - **Bergsteigeralgorithmus mit Random Restart** (pentru Damespiel n x n) - primeste parametru o lista cu un obiect de tip `Damespiel`, reprezentand configuratia (random) initiala


In [0]:
def bombur(node_list):
    step_count = 0
    visited = []
    node_list.sort(key=lambda x: x.cost())
    while len(node_list) != 0 and node_list[0].cost() != 0:
        visited.append(Damenspiel(node_list[0].columns))
        if step_count % 500 == 0:
            print(step_count, node_list[0].cost(), len(visited), node_list[0].columns)
        children = node_list[0].move()
        possibi = list(filter(lambda x: x.cost() <= node_list[0].cost() and x not in visited, children))
        if len(possibi) == 0:
            lista = random.sample(range(n), n)
            instance = Damenspiel(lista)
            node_list = [instance]
            while instance in visited:
                lista = random.sample(range(n), n)
                instance = Damenspiel(lista)
            node_list = [instance]
            print("restarted", step_count, node_list[0].cost())
        else:
            node_list.pop(0)
            possibi.sort(key=lambda x: x.cost())
            node_list = possibi + node_list
        step_count += 1
    return step_count

`ori()` - **Bergsteigeralgorithmus mit simulierten Abkühlung** (pentru Damenspiel n x n) - primeste parametru o lista cu un obiect de tip `Damenspiel`, reprezentand configuratia (random) initiala


In [0]:
def ori(node_list):
    step_count = 1
    visited = []
    node_list.sort(key=lambda x: x.cost())
    while len(node_list) != 0 and node_list[0].cost() != 0:
        visited.append(Damenspiel(node_list[0].columns))
        if step_count % 500 == 0:
            print(step_count, node_list[0].cost(), len(visited), node_list[0].columns)
        temp = 100 / step_count
        children = node_list[0].move()
        possibi = []
        for x in children:
          try:
            prob = math.exp(abs(x.cost() - node_list[0].cost()) / temp)
          except OverflowError:
            prob = 0.5
          if x.cost() <= node_list[0].cost() or random.uniform(0, 1) <=  prob and x not in visited:
            possibi.append(x)
        possibi.sort(key=lambda x:x.cost())
        node_list.pop(0)
        node_list = possibi + node_list
        step_count += 1
    return step_count

Testing the 3 algorithms on the n x n Puzzle Problem:

In [0]:
import random

lista = random.sample(range(n * n), n * n)
while not isSolvable(lista):
    lista = random.sample(range(n * n), n * n)
poz = lista.index(0)
index = 0
matr = []
for i in range(n):
    matr.append([])
    for j in range(n):
        matr[i].append(lista[index])
        index += 1
instance = Puzzle(matr, poz)
node_list = [instance]

# print(bilbo(node_list))
# print(kili(node_list))
# print(fili(node_list))

Testing the 3 algorithms on the n x n Queens Problem:

In [0]:
import random
import math

lista = random.sample(range(n), n)
instance = Damenspiel(lista)

# print(bofur([instance]))
# print(bombur([instance]))
print(ori([instance]))

500 1 500 [7, 5, 2, 4, 6, 0, 3, 1]
1000 1 1000 [7, 5, 2, 4, 6, 0, 3, 1]
1500 1 1500 [7, 5, 2, 4, 6, 0, 3, 1]
2000 1 2000 [7, 5, 2, 4, 6, 0, 3, 1]
2500 1 2500 [7, 5, 2, 4, 6, 0, 3, 1]
3000 1 3000 [7, 5, 2, 4, 6, 0, 3, 1]
3500 1 3500 [7, 5, 2, 4, 6, 0, 3, 1]
4000 1 4000 [7, 5, 2, 4, 6, 0, 3, 1]
4500 1 4500 [7, 5, 2, 4, 6, 0, 3, 1]
